In [16]:
import boto3
from mcp import stdio_client, StdioServerParameters
from strands import Agent
from strands.tools.mcp import MCPClient
import os

# Set global profile name variable that can be used by boto3 and use_aws tool
profile_name = "default"

# Set AWS_PROFILE environment variable globally for all boto3 sessions
os.environ["AWS_PROFILE"] = profile_name
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

# Set default session profile for boto3 (for legacy code and wrapper tools)
boto3.setup_default_session(profile_name=profile_name)

query_mcp_client = MCPClient(lambda: stdio_client(StdioServerParameters(command="uvx", 
    args=["awslabs.amazon-neptune-mcp-server@latest"],
    env={"NEPTUNE_ENDPOINT": "neptune-db://midas-test.cluster-c7j2zglv4rfb.us-east-1.neptune.amazonaws.com"},
    )))

with query_mcp_client:
    tools = query_mcp_client.list_tools_sync()
    # Create an agent with all tools
    agent = Agent(tools=tools, 
        system_prompt="""You are an agent that interacts with an Amazon Neptune database to run graph queries.
        Whenever you write queries you should first fetch the schema to ensure that you understand the correct labels and property names 
        as well as the appropriate casing of those names and values.
        """,
    )
    agent("Tell me about the graph schema")

I'll fetch the graph schema for you to show you the structure of the Neptune database.
Tool #1: get_graph_schema
Based on the graph schema, here's what I can tell you about this Neptune database:

## Graph Overview
This appears to be a biomedical knowledge graph using the **Biolink Model** as its schema framework. The graph contains biological and chemical entities with various relationships between them.

## Node Types (Vertices)
The graph contains **30 different node types**, all prefixed with `biolink:`. The main categories include:

### Chemical Entities:
- `ChemicalEntity`, `ChemicalMixture`, `SmallMolecule`
- `ChemicalEntityOrProteinOrPolypeptide`
- `ChemicalOrDrugOrTreatment`
- `MolecularEntity`, `MolecularMixture`
- `ComplexMolecularMixture`

### Biological Entities:
- `Gene`, `Protein`, `Polypeptide`
- `GeneOrGeneProduct`, `GeneProductMixin`
- `GenomicEntity`, `MacromolecularMachineMixin`

### Anatomical/Cellular:
- `Cell`, `AnatomicalEntity`, `GrossAnatomicalStructure`

### D

In [18]:

with query_mcp_client:
    tools = query_mcp_client.list_tools_sync()
    # Create an agent with all tools
    agent = Agent(tools=tools, 
        system_prompt="""You are an agent that interacts with an Amazon Neptune database to run graph queries.
        Whenever you write queries you should first fetch the schema to ensure that you understand the correct labels and property names 
        as well as the appropriate casing of those names and values.
        """,
    )
    agent("List the top 5 diseases with the most associated drugs in the MIDAS knowledge graph. For each disease, show its name and the number of drugs that treat it. Also provide the cypher query used to generate the result.")

I'll help you find the top 5 diseases with the most associated drugs. First, let me check the graph schema to understand the structure and correct labels.
Tool #1: get_graph_schema
Now I can see the schema. I can see that there are disease nodes labeled as `biolink:Disease` and `biolink:DiseaseOrPhenotypicFeature`, and chemical/drug nodes with various labels like `biolink:ChemicalEntityOrProteinOrPolypeptide`, `biolink:ChemicalEntity`, etc. The relationship that connects drugs to diseases they treat is `biolink:treats_or_applied_or_studied_to_treat`.

Let me create a Cypher query to find the top 5 diseases with the most associated drugs:
Tool #2: run_opencypher_query
Perfect! I have the results. Here are the top 5 diseases with the most associated drugs in the MIDAS knowledge graph:

## Top 5 Diseases with Most Associated Drugs

1. **Inflammation** - 384 drugs
2. **Drug-induced liver injury** - 383 drugs  
3. **Hypertension** - 380 drugs
4. **Pain** - 379 drugs
5. **Visual epilepsy** -